In [1]:
import pandas as pd
import numpy as np
from neighborhoods import madrid_districts_dict
import re

In [2]:
df = pd.read_json('../data/raw/final_scraped_all_22Jul.json')

In [3]:
df.head()

,Title,Price,Url,Bedrooms,Area,Full description,Short description,Features
0,"Piso en calle de Toledo, Acacias, Madrid",209.995€,/inmueble/105157270/,2 hab.,50 m²,! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,"[50 m² construidos, 2 habitaciones, 1 baño, Te..."
1,"Piso en calle de Carlos Arniches, Lavapiés-Emb...",209.995€,/inmueble/105129078/,2 hab.,50 m²,! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,"[50 m² construidos, 2 habitaciones, 1 baño, Te..."
2,"Piso en Villaverde Alto, Madrid",180.000€,/inmueble/105406757/,1 hab.,50 m²,¡Descubre tu nuevo hogar en Villaverde Alto! E...,¡Descubre tu nuevo hogar en Villaverde Alto! E...,"[50 m² construidos, 1 habitación, 1 baño, Plaz..."
3,"Piso en calle de la Ballesta, Malasaña-Univers...",290.000€,/inmueble/103865983/,1 hab.,50 m²,MAGNÍFICO PISO EN VENTA EN LA CONFLUENCIA DE M...,MAGNÍFICO PISO EN VENTA EN LA CONFLUENCIA DE M...,"[50 m² construidos, 45 m² útiles, 1 habitación..."
4,"Piso en calle de Samaniego, Rejas, Madrid",176.000€,/inmueble/104558678/,1 hab.,50 m²,GPHOME VENDE MAGNIFICO APARTAMENTO TURISTICO E...,GPHOME VENDE MAGNIFICO APARTAMENTO TURISTICO E...,"[50 m² construidos, 1 habitación, 1 baño, Plaz..."


In [4]:
df = df[['Price', 'Url', 'Title', 'Full description', 'Bedrooms', 'Features', 'Area']]

In [5]:
df['Price'] = df['Price'].str.replace('.', '').str.replace('€', '').apply(lambda x: int(x))
df['price'] = df['Price'].copy()
df['Price'].describe()

/var/folders/zm/z5q67jgd0vv23h_ypgdpl9ph0000gn/T/ipykernel_83256/3743308868.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('.', '').str.replace('€', '').apply(lambda x: int(x))


count    8.487000e+03
mean     9.673948e+05
std      1.186993e+06
min      2.610000e+04
25%      2.076300e+05
50%      4.950000e+05
75%      1.290000e+06
max      1.500000e+07
Name: Price, dtype: float64

In [6]:
# df['Title'] = df['Title'].str.replace(',', '')
# df['Title'] = df['Title'].apply(lambda x: x.split())
# df['neighborhood'] = df['Title'].apply(lambda x: x[-2])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8487 entries, 0 to 8486
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Price             8487 non-null   int64 
 1   Url               8487 non-null   object
 2   Title             8487 non-null   object
 3   Full description  8439 non-null   object
 4   Bedrooms          8487 non-null   object
 5   Features          8487 non-null   object
 6   Area              8487 non-null   object
 7   price             8487 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 530.6+ KB


In [8]:
df_area_values = []
not_found = []
for index, item in enumerate(df['Features'].values):
    found = False
    for value in item:
        if 'construidos' in value:
            df_area_values.append(value)
            found = True
            break
    if not found:
        if 'm²' in df['Area'].iloc[index]:
            df_area_values.append(df['Area'].iloc[index])
        else:
           df_area_values.append(df['Bedrooms'].iloc[index])

df['m2'] = df_area_values

df.iloc[not_found]

df['m2'] = df['m2'].str.replace('.', '')
df['m2'] = df['m2'].apply(lambda x: float(x.split()[0]))

/var/folders/zm/z5q67jgd0vv23h_ypgdpl9ph0000gn/T/ipykernel_83256/2424817073.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['m2'] = df['m2'].str.replace('.', '')


In [9]:
df['Features'] = df['Features'].apply(lambda x: ', '.join(x))

In [10]:
df_rooms_values = []

for index, value in enumerate(df['Features'].values):
    if 'habitaciones' in value:
        pattern = r'\b(\w+)\s+habitaciones\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        df_rooms_values.append(f'{word_before[0]}')
    elif 'habitación' in value:
        pattern = r'\b(\w+)\s+habitación\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        df_rooms_values.append(f'{word_before[0]}')
    elif 'hab.' in df['Bedrooms'].iloc[index]:
        df_rooms_values.append(df['Bedrooms'].iloc[index].split(' ')[0])
    else:
        df_rooms_values.append('not found')
    
df['rooms'] = df_rooms_values
df = df[df['rooms']!='not found']
df['rooms'] = df['rooms'].replace('Sin', '0').apply(lambda x: int(x))

In [11]:
df_house_type_2 = []

for index, item in enumerate(df['Features'].values):
    found = False
    if 'exterior' in item.lower() and 'interior' in item.lower():
        value = item.lower()
        pattern_ext = r'\b(\w+)\s+exterior\b'
        match_ext = re.search(pattern_ext, value)
        word_before_ext = match_ext.groups()
        pattern_int = r'\b(\w+)\s+interior\b'
        match_int = re.search(pattern_int, value)
        word_before_int = match_int.groups()
        if word_before_int[0] == 'solo':
            df_house_type_2.append('exterior')
        if word_before_ext[0] == 'acceso':
            df_house_type_2.append('interior')
        found = True
    elif 'exterior' in item.lower():
        df_house_type_2.append('exterior')
        found = True
    elif 'interior' in value.lower():
        df_house_type_2.append('interior')
        found = True
    else:
        df_house_type_2.append('not found')

df['house_type_2'] = df_house_type_2


In [12]:
has_garage = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'garage' in value:
        pattern = r'\b(\w+)\s+garage\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        if word_before_int[0] == 'sin':
            has_garage.append(0)
        else:
            has_garage.append(1)
    elif 'garaje' in value:
        pattern = r'\b(\w+)\s+garaje\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        if word_before_int[0] == 'sin':
            has_garage.append(0)
        else:
            has_garage.append(1)
    else:
        has_garage.append(0)

df['garage'] = has_garage

In [13]:
df.head()

,Price,Url,Title,Full description,Bedrooms,Features,Area,price,m2,rooms,house_type_2,garage
0,209995,/inmueble/105157270/,"Piso en calle de Toledo, Acacias, Madrid",! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,2 hab.,"50 m² construidos, 2 habitaciones, 1 baño, Ter...",50 m²,209995,50.0,2,interior,0
1,209995,/inmueble/105129078/,"Piso en calle de Carlos Arniches, Lavapiés-Emb...",! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,2 hab.,"50 m² construidos, 2 habitaciones, 1 baño, Ter...",50 m²,209995,50.0,2,interior,0
2,180000,/inmueble/105406757/,"Piso en Villaverde Alto, Madrid",¡Descubre tu nuevo hogar en Villaverde Alto! E...,1 hab.,"50 m² construidos, 1 habitación, 1 baño, Plaza...",50 m²,180000,50.0,1,exterior,1
3,290000,/inmueble/103865983/,"Piso en calle de la Ballesta, Malasaña-Univers...",MAGNÍFICO PISO EN VENTA EN LA CONFLUENCIA DE M...,1 hab.,"50 m² construidos, 45 m² útiles, 1 habitación,...",50 m²,290000,50.0,1,interior,0
4,176000,/inmueble/104558678/,"Piso en calle de Samaniego, Rejas, Madrid",GPHOME VENDE MAGNIFICO APARTAMENTO TURISTICO E...,1 hab.,"50 m² construidos, 1 habitación, 1 baño, Plaza...",50 m²,176000,50.0,1,exterior,1


In [14]:
has_elevator = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'ascensor' in value:
        pattern = r'\b(\w+)\s+ascensor\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        if word_before_int[0] == 'sin':
            has_elevator.append(0)
        else:
            has_elevator.append(1)
    else:
        has_elevator.append(0)

df['elevator'] = has_elevator

In [15]:
df['house_type'] = df['Title'].apply(lambda x: x.split(' en ')[0])

df['house_type'] = df['house_type'].replace('Chalet adosado', 'Chalet').replace('Chalet pareado', 'Chalet')

df['house_type'] = df['house_type'].replace('Casa o chalet independiente', 'Casa')

In [16]:
floor_num = []
not_found = []

for index, item in enumerate(df['Features'].values):
    found = False
    item = item.lower()
    item = item.split(', ')
    for value in item:
        if 'planta' in value and 'entreplanta' not in value:
            value = value.replace(' plantas', '').replace(' planta', '').replace('planta ', '').replace(' exterior', '').replace(' interior', '').replace('ª', '')
            floor_num.append(value)
            found = True
            break
    if not found:
        not_found.append(index)
        floor_num.append(0) # floor not specified

df['floor'] = floor_num
df['floor'] = df['floor'].apply(lambda x: int(x))

In [17]:
madrid_neighborhoods = []
madrid_districts = []

for key in madrid_districts_dict:
    madrid_neighborhoods.extend(madrid_districts_dict[key])
    madrid_districts.extend(key)


In [18]:
df['neighbourhood'] = df['Title'].apply(lambda x: x.split(', ')[-2]).apply(lambda x: x.split(' en ')[-1])

In [19]:
item_conversion_dict = {'Águilas': 'Las Águilas',
 'Palos de la Frontera': 'Palos de Moguer',
 'Malasaña-Universidad': 'Universidad',
 'Valdebebas - Valdefuentes': 'Valdefuentes',
 'Montecarmelo': 'El Goloso',
 'San Cristóbal': 'San Cristóbal de los Ángeles',
 'Delicias': 'Las Delicias',
 'Sanchinarro': 'Valdefuentes',
 'Acacias': 'Las Acacias',
 'Ventilla-Almenara': 'Almenara',
 'Las Tablas': 'Valverde',
 '12 de Octubre-Orcasur': 'Orcasur',
 'Ensanche de Vallecas - La Gavia': 'Casco Histórico de Vallecas',
 'Campo de las Naciones-Corralejos': 'Corralejos',
 'Palomeras sureste': 'Palomeras Sureste',
 'Conde Orgaz-Piovera': 'Piovera',
 'Nuevos Ministerios-Ríos Rosas': 'Ríos Rosas',
 'Pau de Carabanchel': 'Buenavista',
 'Pilar': 'Barrio del Pilar',
 'Tres Olivos - Valverde': 'Valverde',
 'Huertas-Cortes': 'Cortes',
 'Arroyo del Fresno': 'Mirasierra',
 'Cuzco-Castillejos': 'Castillejos',
 'Valdebernardo - Valderrivas': 'Valdebernardo',
 'Chueca-Justicia': 'Justicia',
 'Zofío': 'Zofio',
 'Lavapiés-Embajadores': 'Embajadores',
 'Buena Vista': 'Buenavista',
 'Chopera': 'La Chopera',
 'Virgen del Cortijo - Manoteras': 'Valdefuentes',
 'Costillares': 'Pinar de Chamartín',
 'Bernabéu-Hispanoamérica': 'Hispanoamérica'
 }

In [20]:
for key in item_conversion_dict:
    df['neighbourhood'] = df['neighbourhood'].replace(key, item_conversion_dict[key])

In [21]:
df['district'] = df['neighbourhood'].copy()

In [22]:
for barrio in set(df['district'].values):
    for district in madrid_districts_dict:
        if barrio in madrid_districts_dict[district]:
            df['district'] = df['district'].replace(barrio, district)

In [23]:
df = df[['price', 'house_type', 'house_type_2', 'rooms', 'm2', 'elevator',
       'garage', 'neighbourhood', 'district']]

In [24]:
df.head()

,price,house_type,house_type_2,rooms,m2,elevator,garage,neighbourhood,district
0,209995,Piso,interior,2,50.0,1,0,Las Acacias,Arganzuela
1,209995,Piso,interior,2,50.0,1,0,Embajadores,Centro
2,180000,Piso,exterior,1,50.0,1,1,Villaverde Alto,Villaverde
3,290000,Piso,interior,1,50.0,1,0,Universidad,Centro
4,176000,Piso,exterior,1,50.0,1,1,Rejas,San Blas-Canillejas


In [25]:
df.to_csv('../data/processed/scraped_data_cleaned_Emanuela_22Jul24.csv', index=False)